In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = 'PCI_BUS_ID'
os.environ["CUDA_VISIBLE_DEVICES"] = '0'

In [2]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Concatenate,Dense,Input,Embedding,Bidirectional,LSTM,GRU
import keras.backend as K
from tensorflow.keras.utils import to_categorical

In [3]:
import transformers

In [4]:
imdb_df = pd.read_csv("/data2/scratch/rohitr/datasets/IMDB_Dataset.csv")

In [5]:
d = {'negative':0,'positive':1}

In [6]:
imdb_df['sentiment'] = imdb_df['sentiment'].apply(lambda x : d.get(x))

In [7]:
imdb_df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [8]:
X_train,X_test,y_train,y_test = train_test_split(imdb_df['review'],imdb_df['sentiment'],test_size=0.2)

In [10]:
from transformers import AutoTokenizer,TFBertModel
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
bert = TFBertModel.from_pretrained("bert-base-cased")

2022-09-02 20:04:30.816825: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-09-02 20:04:33.894027: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38418 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:07:00.0, compute capability: 8.0


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

2022-09-02 20:05:36.543620: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further traini

In [11]:
# Tokenize the input (takes some time) 
# here tokenizer using from bert-base-cased
x_train = tokenizer(
    text=X_train.tolist(),
    add_special_tokens=True,
    max_length=128,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)
x_test = tokenizer(
    text=X_test.tolist(),
    add_special_tokens=True,
    max_length=128,
    truncation=True,
    padding=True, 
    return_tensors='tf',
    return_token_type_ids = False,
    return_attention_mask = True,
    verbose = True)

In [13]:
input_ids = x_train['input_ids']
attention_mask = x_train['attention_mask']

In [14]:
max_len = 128
input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
input_mask = Input(shape=(max_len,), dtype=tf.int32, name="attention_mask")
embeddings = bert(input_ids,attention_mask = input_mask)[0] 
out = tf.keras.layers.GlobalMaxPool1D()(embeddings)
out = Dense(128, activation='relu')(out)
out = tf.keras.layers.Dropout(0.1)(out)
out = Dense(32,activation = 'relu')(out)
y = Dense(1,activation = 'sigmoid')(out)
model = tf.keras.Model(inputs=[input_ids, input_mask], outputs=y)
model.layers[2].trainable = True

In [16]:


from tensorflow.keras.optimizers import Adam
optimizer = Adam(
    learning_rate=5e-05, # this learning rate is for bert model , taken from huggingface website 
    epsilon=1e-08,
    decay=0.01,
    clipnorm=1.0)

In [18]:
# loss = CategoricalCrossentropy(from_logits = True)

In [20]:
# Compile the model
model.compile(
    optimizer = optimizer,
    loss = 'binary_crossentropy', 
    metrics = ['acc'])

In [21]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, y_test
    ),
  epochs=2,
    batch_size=64
)

2022-09-02 20:11:37.661764: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/2
625/625 [==============================] - 211s 233ms/step - loss: 0.3359 - acc: 0.8504 - val_loss: 0.2786 - val_acc: 0.8804
Epoch 2/2
625/625 [==============================] - 131s 210ms/step - loss: 0.2292 - acc: 0.9069 - val_loss: 0.2762 - val_acc: 0.8840


In [22]:
train_history = model.fit(
    x ={'input_ids':x_train['input_ids'],'attention_mask':x_train['attention_mask']} ,
    y = y_train,
    validation_data = (
    {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, y_test
    ),
  epochs=2,
    batch_size=64
)

Epoch 1/2
625/625 [==============================] - 131s 210ms/step - loss: 0.1903 - acc: 0.9267 - val_loss: 0.2788 - val_acc: 0.8855
Epoch 2/2
625/625 [==============================] - 131s 210ms/step - loss: 0.1673 - acc: 0.9362 - val_loss: 0.2925 - val_acc: 0.8848


In [23]:
predicted_raw = model.predict({'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']})
predicted_raw[0]

array([0.00948634], dtype=float32)

In [24]:
model.evaluate(x = {'input_ids':x_test['input_ids'],'attention_mask':x_test['attention_mask']}, y = y_test)

313/313 [==============================] - 30s 95ms/step - loss: 0.2925 - acc: 0.8848


[0.2925480306148529, 0.8848000168800354]

In [ ]:
y_predicted = np.argmax(predicted_raw, axis = 1)
#y_true = df_test.Sentiment

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_predicted))